# core

> Create messages for language models like Claude and OpenAI GPTs (including Azure OpenAI).

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import base64
import mimetypes
from collections.abc import Mapping

from fastcore import imghdr
from fastcore.meta import delegates
from fastcore.utils import *

In [ ]:
from IPython.display import Image, display
from pathlib import Path

In [ ]:
from pathlib import Path

## API Exploration

Anthropic's Claude and OpenAI's GPT models are some of the most popular LLMs. 

Let's take a look at their APIs and to learn how we should structure our messages for a simple text chat.

#### openai

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

client.responses.create(
  model="gpt-4.1",
  input=[ {"role": "user", "content": "Hello, world!"} ]
)

Response(id='resp_0e31fb3c9c3089d500693937a77a6c8194889e95247c1f316d', created_at=1765357479.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_0e31fb3c9c3089d500693937a8122c81948f695ad3f5cdc081', content=[ResponseOutputText(annotations=[], text='Hello, world! 🌍  \nHow can I help you today?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, prompt_cache_retention=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), top_logprobs=0, truncation=

#### azure/openai

In [ ]:
from openai import AzureOpenAI

In [ ]:
az_res = client.chat.completions.create(
    model="Gpt4.1",
    messages=[ {"role": "user", "content": "Hello, world!"} ]
)
az_res

ChatCompletion(id='chatcmpl-ClAR7iIxGFYHPyjERTnZ4R8FIzHmD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! 🌍 How can I help you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1765357569, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_f99638a8d7', usage=CompletionUsage(completion_tokens=12, prompt_tokens=11, total_tokens=23, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_pre

#### anthropic

In [ ]:
from anthropic import Anthropic

In [ ]:
client = Anthropic()

client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    messages=[ {"role": "user", "content": "Hello, world!"} ]
)

Message(id='msg_018eV3E7midA69EEb6EvGuPr', content=[TextBlock(citations=None, text="Hello! It's nice to meet you. As an AI assistant, I'm here to help with any questions or tasks you may have. Please feel free to ask me anything, and I'll do my best to assist you.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=11, output_tokens=49, server_tool_use=None, service_tier='standard'))

As we can see the APIs use the exact same message structure.

### mk_msg

Ok, let's build the first version of `mk_msg` to handle this case

In [ ]:
def mk_msg(content:str, role:str="user")->dict:
    "Create an OpenAI/Anthropic compatible message."
    return dict(role=role, content=content)

Let's test it out with the OpenAI API. To do that we'll need to setup two things:

- install the openai SDK by running `pip install openai`
- add your openai api key to your env vars `export OPENAI_API_KEY="YOUR_OPEN_API_KEY"`

In [ ]:
oa_cli = OpenAI()

r = oa_cli.responses.create(
  model="gpt-4o-mini",
  input=[mk_msg("Hello, world!")]
)
r.output_text

'Hello! How can I assist you today?'

For the Azure/OpenAI API, the Python openai module provides the required class, but we must set the following environment variables:

- `export AZURE_OPENAI_API_KEY="YOUR_API_KEY"`
- `export OPENAI_API_VERSION="API_VERSION"`
- `export AZURE_OPENAI_ENDPOINT="ENDPOINT_URL"`

In [ ]:
az_cli = AzureOpenAI()

r = az_cli.chat.completions.create(
    model="Gpt4.1",
    messages=[mk_msg("Hello, world!")]
)

r.choices[0].message.content

'Hello! 🌍 How can I help you today?'

Now, let's test out `mk_msg` on the Anthropic API. To do that we'll need to setup two things:

- install the openai SDK by running `pip install anthropic`
- add your anthropic api key to your env vars `export ANTHROPIC_API_KEY="YOUR_ANTHROPIC_API_KEY"`

In [ ]:
a_cli = Anthropic()

r = a_cli.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    messages=[mk_msg("Hello, world!")]
)
r.content[0].text

"Hello! I'm an AI assistant created by Anthropic. How can I assist you today?"

So far so good!

#### Helper Functions

Before going any further, let's create some helper functions to make it a little easier to call the OpenAI and Anthropic APIs. We're going to be making a bunch of API calls to test our code and typing the full expressions out each time will become a little tedious. These functions won't be included in the final package.

In [ ]:
def openai_chat(msgs: list)->tuple:
    "call the openai chat responses endpoint with `msgs`."
    r = oa_cli.responses.create(model="o4-mini", input=msgs)
    return r, r.output_text

Let's double check that `mk_msg` still works with our simple text example from before.

In [ ]:
_, text = openai_chat([mk_msg("Hello, world!")])
text

'Hello there! How can I assist you today?'

and Azure/OpenAI...

In [ ]:
def azure_chat(msgs:list)->tuple:
    "call the azure/openai messages endpoint with `msgs`."
    r = az_cli.chat.completions.create(model="Gpt4.1", messages=msgs)
    return r, r.choices[0].message.content

In [ ]:
_, text = azure_chat([mk_msg("Hello, world!")])
text

'Hello! 🌍 How can I assist you today?'

and Anthropic...

In [ ]:
a_cli = Anthropic()

def anthropic_chat(msgs: list)->tuple:
    "call the anthropic messages endpoint with `msgs`."
    r = a_cli.messages.create(model="claude-sonnet-4-20250514", max_tokens=1024, messages=msgs)
    return r, r.content[0].text

In [ ]:
_, text = anthropic_chat([mk_msg("Hello, world!")])
text

'Hello! Nice to meet you! How are you doing today? Is there anything I can help you with?'

### Images

Ok, let's see how the APIs handle image messages.

<img src="https://claudette.answer.ai/index_files/figure-html/cell-35-output-1.jpeg" height=240 width=240></img>

#### openai

In [ ]:
import base64, httpx

In [ ]:
img_url = "https://claudette.answer.ai/index_files/figure-html/cell-35-output-1.jpeg"

In [ ]:
mtype = "image/jpeg"
img_content = httpx.get(img_url).content

In [ ]:
img = base64.b64encode(img_content).decode("utf-8")

client = OpenAI()
r = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role":"user",
            "content": [
                {"type":"input_text","text":"What's in this image?"},
                {"type":"input_image","image_url":f"data:image/jpeg;base64,{img}"},
            ],
        }
    ],
)
r.output_text

'The image features a cute puppy lying on the grass near some purple flowers. The puppy has a white coat with brown patches and appears inquisitive and playful.'

#### azure/openai

In [ ]:
mtype = "image/jpeg"
img = base64.b64encode(img_content).decode("utf-8")

r = az_cli.chat.completions.create(
    model="Gpt4.1",
    messages=[
        {
            "role":"user",
            "content": [
                {"type":"text","text":"What's in this image?"},
                {"type":"image_url","image_url": {'url': f"data:image/jpeg;base64,{img}"}},
            ],
        }
    ],
)
r.choices[0].message.content

'The image shows a small dog with white fur and brown ears lying on the grass next to a bunch of purple flowers. There is also a wooden object in the background.'

#### anthropic

In [ ]:
mtype = "image/jpeg"
img = base64.b64encode(img_content).decode("utf-8")

client = Anthropic()
r = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    messages=[
        {
            "role":"user",
            "content": [
                {"type":"text","text":"What's in this image?"},
                {"type":"image","source":{"type":"base64","media_type":mtype,"data":img}}
            ],
        }
    ],
)
r.content[0].text

'This image shows a cute puppy lying on the grass. The puppy appears to be a Cavalier King Charles Spaniel, with a long, silky coat that is brown and white in color. The puppy has a sweet, friendly expression as it gazes out at the viewer. In the background, there are some purple flowers, likely daisies or asters, adding a nice natural element to the scene.'

ALl three APIs format images slightly differently and the structure of the message `content` is a little more complex. 

In a text chat, `content` is a simple string but for a multimodal chat (text+images) we can see that `content` is a list of dictionaries.

## Msg Class

### Basics

Let's create `_mk_img` to make our code a little DRY'r. 

In [ ]:
#| exports
def _mk_img(data:bytes)->tuple:
    "Convert image bytes to a base64 encoded image"
    img = base64.b64encode(data).decode("utf-8")
    mtype = mimetypes.types_map["."+imghdr.what(None, h=data)]
    return img, mtype

To handle the additional complexity of multimodal messages let's build a `Msg` class for the `content` data structure:

```json
{
    "role": "user",
    "content": [{"type": "text", "text": "What's in this image?"}],
}
```

In [ ]:
#| exports
class Msg:
    "Helper class to create a message for the OpenAI and Anthropic APIs."
    pass

As all three APIs handle images differently let's subclass `Msg` for each API and handle the image formatting in a method called `img_msg`.

In [ ]:
#| exports
class OpenAiMsg(Msg):
    "Helper class to create a message for the OpenAI API."
    pass

In [ ]:
#| exports
class AzureMsg(Msg):
    "Helper class to create a message for the Azure OpenAI API."
    pass

In [ ]:
#| exports
class AnthropicMsg(Msg):
    "Helper class to create a message for the Anthropic API."
    pass

Let's write some helper functions for `mk_content` to use.

In [ ]:
#| exports
def _is_img(data): return isinstance(data, bytes) and bool(imghdr.what(None, data))

A PDF [file](https://docs.fileformat.com/pdf/#pdf-file-header) should start with `%PDF` followed by the pdf version `%PDF-1.1`

In [ ]:
#| exports
def _is_pdf(data): 
    is_byte_pdf = isinstance(data, bytes) and data.startswith(b'%PDF-')
    is_pdf_url = isinstance(data, str) and (
        data.startswith("http") and (data.endswith(".pdf") or 'pdf' in data.split('/'))
    )
    return is_byte_pdf or is_pdf_url

In [ ]:
assert _is_pdf("https://arxiv.org/pdf/2301.00001")
assert not _is_pdf("https://arxiv.org/abs/2301.00001")
assert _is_pdf("https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf")
assert not _is_pdf("Hi /pdf/")

We create an appropriate type based on content:

In [ ]:
#| exports
@patch
def mk_content(self:Msg, content, text_only=False)->dict:
    if _is_img(content): return self.img_msg(content)
    if _is_pdf(content): return self.pdf_msg(content)
    if isinstance(content, str): return self.text_msg(content, text_only=text_only)
    return content

…then we call the model with this content:

In [ ]:
@patch
def __call__(self:Msg, role:str, content:[list, str], text_only:bool=False, **kw)->dict:
    "Create an OpenAI/Anthropic compatible message with `role` and `content`."
    if content is not None and not isinstance(content, list): content = [content]
    content = [self.mk_content(o, text_only=text_only) for o in content] if content else ''
    return dict(role=role, content=content[0] if text_only else content, **kw)

OpenAI implementations:

In [ ]:
#| exports
@patch
def img_msg(self:OpenAiMsg, data:bytes)->dict:
    "Convert `data` to an image message"
    img, mtype = _mk_img(data)
    return {"type": "input_image", "image_url": f"data:{mtype};base64,{img}"}

@patch
def text_msg(self:OpenAiMsg, s:str, text_only=False)->dict: 
    "Convert `s` to a text message"
    if not s.strip(): s='.'
    return s if text_only else {"type": "input_text", "text":s}

Azure/OpenAI implementations:

In [ ]:
#| exports
@patch
def img_msg(self:AzureMsg, data:bytes)->dict:
    "Convert `data` to an image message"
    img, mtype = _mk_img(data)
    return {"type": "image_url", "image_url": {'url': f"data:{mtype};base64,{img}"}}

@patch
def text_msg(self:AzureMsg, s:str, text_only=False)->dict: 
    "Convert `s` to a text message"
    if not s.strip(): s='.'
    return s if text_only else {"type": "text", "text":s}

Anthropic implementations:

In [ ]:
#| exports
@patch
def img_msg(self:AnthropicMsg, data:bytes)->dict:
    "Convert `data` to an image message"
    img, mtype = _mk_img(data)
    r = {"type": "base64", "media_type": mtype, "data":img}
    return {"type": "image", "source": r}

@patch
def text_msg(self:AnthropicMsg, s:str, text_only=False)->dict: 
    "Convert `s` to a text message"
    if not s.strip(): s='.'
    return s if text_only else {"type": "text", "text":s}

Update `mk_msg` to use `Msg`.

In [ ]:
#| export
PROVIDER_MSG = {
    'openai': OpenAiMsg,
    'azure': AzureMsg,
    'anthropic': AnthropicMsg,
}

In [ ]:
#| export
def mk_msg(content:Union[list,str], role:str="user", *args, api:str="openai", **kw)->dict:
    "Create an OpenAI/Anthropic compatible message."
    text_only = isinstance(content, str) or (isinstance(content, list) and len(content) == 1 and isinstance(content[0], str))
    m = PROVIDER_MSG[api]
    msg = m()(role, content, text_only=text_only, **kw)
    return dict2obj(msg, list_func=list)

In [ ]:
mk_msg(["Hello world", "how are you?"], api='openai')

```python
{ 'content': [ {'text': 'Hello world', 'type': 'input_text'},
               {'text': 'how are you?', 'type': 'input_text'}],
  'role': 'user'}
```

In [ ]:
mk_msg(["Hello world", "how are you?"], api='azure')

```python
{ 'content': [ {'text': 'Hello world', 'type': 'text'},
               {'text': 'how are you?', 'type': 'text'}],
  'role': 'user'}
```

In [ ]:
mk_msg(["Hello world", "how are you?"], api='anthropic')

```python
{ 'content': [ {'text': 'Hello world', 'type': 'text'},
               {'text': 'how are you?', 'type': 'text'}],
  'role': 'user'}
```

In [ ]:
msg = mk_msg([img_content, "describe this picture"], api="openai")
_, text = openai_chat([msg])
text

'The image shows a small puppy—likely a Cavalier King Charles Spaniel—resting on a patch of green grass. Its fur is predominantly white, with rich chestnut-brown patches on its floppy ears and around its eyes. The puppy’s dark, round eyes gaze directly at the camera, giving it an inquisitive, gentle expression. Behind it, there’s a cluster of small purple daisy-like flowers and a wooden surface (perhaps a low bench or planter). Soft, natural light illuminates the scene, highlighting the puppy’s silky coat and the vibrant colors of the grass and blossoms.'

In [ ]:
msg = mk_msg([img_content, "describe this picture"], api="azure")
_, text = azure_chat([msg])
text

'A dog with a white body and brown ears is lying on the grass near a bunch of purple flowers. The background includes greenery and a rustic-looking surface. The scene appears peaceful and outdoorsy, with the dog relaxing in a garden setting.'

In [ ]:
msg = mk_msg([img_content, "describe this picture"], api="anthropic")
_, text = anthropic_chat([msg])
text

"This adorable image shows a young puppy, likely a Cavalier King Charles Spaniel, sitting in a garden setting. The puppy has beautiful reddish-brown and white fur with distinctive markings - a white blaze down the center of its face and white chest, contrasted with rich brown coloring around the ears and eyes. The puppy has sweet, dark eyes and appears to be quite young.\n\nThe setting is charming, with the puppy positioned near some purple flowers (possibly asters or similar small blooms) and sitting on what appears to be green grass or moss. There's a rustic brick or stone wall visible in the background, creating a lovely garden atmosphere. The lighting appears soft and natural, giving the whole scene a warm, peaceful quality that perfectly complements the puppy's gentle expression."

### PDFs

What about chatting with PDFs? Unfortunately, OpenAI's message completions API doesn't offer PDF support at the moment, but Claude [does](https://docs.anthropic.com/en/docs/build-with-claude/pdf-support). 

Under the hood, Claude extracts the text from the PDF and converts each page to an image. This means you can ask Claude about any text, pictures, charts, and tables in the PDF. Here's an example from the Claude [docs](https://docs.anthropic.com/en/docs/build-with-claude/pdf-support#how-to-use-pdfs-in-the-messages-api). Overall the message structure is pretty similar to an image message.

```python
pdf_url = "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"
pdf_data = base64.standard_b64encode(httpx.get(pdf_url).content).decode("utf-8")
client = anthropic.Anthropic()
message = client.messages.create(
    model="claude-3-5-sonnet-20241022", max_tokens=1024,
    messages=[{
        "role": "user",
        "content": [
            {
                "type": "document",
                "source": { "type": "base64", "media_type": "application/pdf", "data": pdf_data }
            },
            {
                "type": "text",
                "text": "Which model has the highest human preference win rates across each use-case?"
            }
        ]
    }]
)
```

The Anthropic API has since offered an option for PDFs that can be accessed online via url.

```python
client = anthropic.Anthropic()
message = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "url",
                        "url": "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"
                    }
                },
                {
                    "type": "text",
                    "text": "What are the key findings in this document?"
                }
            ]
        }
    ],
)
```

Let's create a method that converts a byte string to the base64 encoded string that Anthropic expects.

In [ ]:
#| exports
def _mk_pdf(data:bytes)->str:
    "Convert pdf bytes to a base64 encoded pdf"
    return base64.standard_b64encode(data).decode("utf-8")

We add a `pdf_msg` method to `AnthropicMsg` that uses `_mk_pdf`.

In [ ]:
#| exports
@patch
def pdf_msg(self:AnthropicMsg, data: bytes | str) -> dict:
    "Convert `data` to a pdf message"
    if isinstance(data, bytes):
        r = {"type": "base64", "media_type": "application/pdf", "data":_mk_pdf(data)}
    elif isinstance(data, str):
        r = {"type": "url", "url": data}
    return {"type": "document", "source": r}


Let's test our changes on a financial report.

In [ ]:
pdf = Path('financial_report.pdf').read_bytes()
msg = mk_msg([pdf, "what was the average monthly revenue for product D?"], api="anthropic")
_, text = anthropic_chat([msg])
text

In [ ]:
pdf_url = "https://arxiv.org/pdf/2506.18880"
msg = mk_msg([pdf_url, "What were the three types of generalization the authors of this paper looked at?"], api="anthropic")
_, text = anthropic_chat([msg])
text

### Conversation

LLMs are stateless. To continue a conversation we need to include the entire message history in every API call.
By default the role in each message alternates between `user` and `assistant`.

Let's add a method that alternates the roles for us and then calls `mk_msgs`.

In [ ]:
def mk_msgs(msgs: list, *args, api:str="openai", **kw) -> list:
    "Create a list of messages compatible with OpenAI/Anthropic."
    if isinstance(msgs, str): msgs = [msgs]
    return [mk_msg(o, ('user', 'assistant')[i % 2], *args, api=api, **kw) for i, o in enumerate(msgs)]

In [ ]:
mk_msgs(["Hello", "Some assistant response", "tell me a joke"])

[{'role': 'user', 'content': 'Hello'},
 {'role': 'assistant', 'content': 'Some assistant response'},
 {'role': 'user', 'content': 'tell me a joke'}]

### SDK Objects

To make our lives even easier, it would be nice if `mk_msg` could format the SDK objects returned from a previous chat so that we can pass them straight to `mk_msgs`.

The OpenAI SDK accepts objects like `ChatCompletion` as messages. Anthropic is different and expects every message to have the `role`, `content` format that we've seen so far.

In [ ]:
#| export
@patch
def __call__(self:Msg, role:str, content:[list,str], text_only:bool=False, **kw)->dict:
    "Create an OpenAI/Anthropic compatible message with `role` and `content`."
    if self.sdk_obj_support and self.is_sdk_obj(content): return self.find_block(content)
    if hasattr(content, "content"): content, role = content.content, content.role
    content = self.find_block(content)
    if content is not None and not isinstance(content, list): content = [content]
    content = [self.mk_content(o, text_only=text_only) for o in content] if content else ''
    return dict(role=role, content=content[0] if text_only else content, **kw)

In [ ]:
#| export
AnthropicMsg.sdk_obj_support=False
OpenAiMsg.sdk_obj_support=True
AzureMsg.sdk_obj_support=False

In [ ]:
#| export
@patch
def is_sdk_obj(self:AnthropicMsg, r)-> bool:
    "Check if `r` is an SDK object."
    return isinstance(r, Mapping)

@patch
def find_block(self:AnthropicMsg, r):
    "Find the message in `r`."
    return r.get('content', r) if self.is_sdk_obj(r) else r

In [ ]:
#| export
@patch
def is_sdk_obj(self:OpenAiMsg, r)-> bool:
    "Check if `r` is an SDK object."
    return not isinstance(r, (str,bytes,list))

@patch
def find_block(self:OpenAiMsg, r):
    "Find the message in `r`."
    if isinstance(r,Mapping): return r
    if hasattr(r, "output"): return r.output
    return r

In [ ]:
#| export
@patch
def is_sdk_obj(self:AzureMsg, r)-> bool:
    "Check if `r` is an SDK object."
    return not isinstance(r, (str,bytes,list))

@patch
def find_block(self:AzureMsg, r):
    "Find the message in `r`."
    return getattr(r.choices[0].message, 'content', r) if self.is_sdk_obj(r) else r

In [ ]:
#| export
def mk_msgs(msgs: list, *args, api:str="openai", **kw) -> list:
    "Create a list of messages compatible with OpenAI/Anthropic."
    if isinstance(msgs, str): msgs = [msgs]
    mm = [mk_msg(o, ('user', 'assistant')[i % 2], *args, api=api, **kw) for i, o in enumerate(msgs)]
    res = []
    for o in mm:
        if isinstance(o,list): res += o
        else: res.append(o)
    return res

Let's test our changes with OpenAI.

In [ ]:
msgs = ["tell me a joke"]
r, text = openai_chat(mk_msgs(msgs))
text

'Why don’t scientists trust atoms?  \nBecause they make up everything!'

In [ ]:
msgs += [r, "tell me another joke that's similar to your first joke"]
mm = mk_msgs(msgs)
mm

[{'role': 'user', 'content': 'tell me a joke'},
 ResponseReasoningItem(id='rs_074b10d3bc394aed0069393beb28008196afd1dfe4b20684fc', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseOutputMessage(id='msg_074b10d3bc394aed0069393bec44a88196bb8d3565b6ffa290', content=[ResponseOutputText(annotations=[], text='Why don’t scientists trust atoms?  \nBecause they make up everything!', type='output_text', logprobs=[])], role='assistant', status='completed', type='message'),
 {'role': 'user',
  'content': "tell me another joke that's similar to your first joke"}]

In [ ]:
r, text = openai_chat(mm)
text

'Why did the photon refuse to check its luggage?  \nBecause it was traveling light!'

and Azure/OpenAI ...

In [ ]:
msgs = ["tell me a joke"]
r, text = azure_chat(mk_msgs(msgs, api='azure'))
text

'Sure! Here you go:\n\nWhy did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

In [ ]:
msgs += [r, "tell me another joke that's similar to your first joke"]
mm = mk_msgs(msgs, api='azure')
mm

[{'role': 'user', 'content': 'tell me a joke'},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'Sure! Here you go:\n\nWhy did the scarecrow win an award?\n\nBecause he was outstanding in his field!'}]},
 {'role': 'user',
  'content': "tell me another joke that's similar to your first joke"}]

In [ ]:
r, text = azure_chat(mm)
text

'Absolutely! Here’s another in the same vein:\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

and Anthropic ...

In [ ]:
msgs = ["tell me a joke"]
r, text = anthropic_chat(mk_msgs(msgs, api='anthropic'))
text

"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [ ]:
msgs += [r, "tell me another joke that's similar to your first joke"]
mm = mk_msgs(msgs, api='anthropic')
mm

[{'role': 'user', 'content': 'tell me a joke'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="Why don't scientists trust atoms?\n\nBecause they make up everything!", type='text')]},
 {'role': 'user',
  'content': "tell me another joke that's similar to your first joke"}]

In [ ]:
r, text = anthropic_chat(mm)
text

"Why don't electrons ever pay their bills?\n\nBecause they're always negatively charged!"

### Usage

To make `msglm` a little easier to use let's create OpenAI, Azure/OpenAI and Anthropic wrappers for `mk_msg` and `mk_msgs`.

In [ ]:
#| export
mk_msg_openai = partial(mk_msg, api="openai")
mk_msgs_openai = partial(mk_msgs, api="openai")
mk_msg_azure = partial(mk_msg, api="azure")
mk_msgs_azure = partial(mk_msgs, api="azure")

In [ ]:
mk_msg_anthropic = partial(mk_msg, api="anthropic")
mk_msgs_anthropic = partial(mk_msgs, api="anthropic")

If you're using OpenAI you should be able to use the import below

```python
from msglm import mk_msg_openai as mk_msg, mk_msgs_openai as mk_msgs
```

Similarily for Azure/OpenAI

```python
from msglm import mk_msg_azure as mk_msg, mk_msgs_azure as mk_msgs
```

Similarily for Anthropic

```python
from msglm import mk_msg_anthropic as mk_msg, mk_msgs_anthropic as mk_msgs
```

## Extra features

### Caching

Anthropic currently offers [prompt caching](https://docs.anthropic.com/en/docs/build-with-claude/prompt-caching), which can reduce cost and latency.

To cache a message, we simply add a `cache_control` field to our content as shown below.

```js
{
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": "Hello, can you tell me more about the solar system?",
            "cache_control": {"type": "ephemeral"}
        }
    ]
}
```

Let's update our `mk_msg` and `mk_msgs` Anthropic wrappers to support caching.

In [ ]:
#| export
def _add_cache_control(msg, cache=False, ttl=None):
    "cache `msg` with optional ttl."
    if not cache: return msg
    if isinstance(msg["content"], str): msg["content"] = [{"type": "text", "text": msg["content"]}]
    cache_control = {"type": "ephemeral"}
    if ttl is not None: cache_control["ttl"] = ttl
    if isinstance(msg["content"][-1], dict): msg["content"][-1]["cache_control"] = cache_control
    elif isinstance(msg["content"][-1], abc.Mapping): msg["content"][-1].cache_control = cache_control
    return msg

def _remove_cache_ckpts(msg):
    "remove unecessary cache checkpoints."
    if isinstance(msg["content"], str): msg["content"] = [{"type": "text", "text": msg["content"]}]
    elif isinstance(msg["content"][-1], dict): msg["content"][-1].pop('cache_control', None)
    else: delattr(msg["content"][-1], 'cache_control') if hasattr(msg["content"][-1], 'cache_control') else None
    return msg

@delegates(mk_msg)
def mk_msg_anthropic(*args, cache=False, ttl=None, **kwargs):
    "Create an Anthropic compatible message."
    msg = partial(mk_msg, api="anthropic")(*args, **kwargs)
    return _add_cache_control(msg, cache=cache, ttl=ttl)

@delegates(mk_msgs)
def mk_msgs_anthropic(*args, cache=False, ttl=None, cache_last_ckpt_only=False, **kwargs):
    "Create a list of Anthropic compatible messages."
    msgs = partial(mk_msgs, api="anthropic")(*args, **kwargs)
    if cache_last_ckpt_only: msgs = [_remove_cache_ckpts(m) for m in msgs]
    if not msgs: return msgs
    msgs[-1] = _add_cache_control(msgs[-1], cache=cache, ttl=ttl)
    return msgs

Let's see caching in action

In [ ]:
mk_msg_anthropic("Don't cache my message")

In [ ]:
mk_msg_anthropic("Please cache my message", cache=True)

In [ ]:
mk_msg_anthropic("Cache for 1 hour", cache=True, ttl="1h")

### Citations

The Anthropic API provides detailed [citations](https://docs.anthropic.com/en/docs/build-with-claude/citations) when answering questions about documents.

When citations are enabled a citations block like the one below will be included in the response.

```js
{
  "content": [
    { "type": "text", "text": "According to the document, " },
    {
      "type": "text", "text": "the grass is green",
      "citations": [{
        "type": "char_location",
        "cited_text": "The grass is green.",
        "document_index": 0, "document_title": "Example Document",
        "start_char_index": 0, "end_char_index": 20
      }]
    }
  ]
}
```

To enable citations you need to create an Anthropic document with the following structure.

```js
{
    "type": "document",
    "source": {...},
    "title": "Document Title", # optional
    "context": "Context about the document that will not be cited from", # optional
    "citations": {"enabled": True}
}
```

Currently Anthropic supports citations on 3 document types:
- text
- pdfs
- custom

A **text** document has the following source structure.

```js
{"type": "text", "media_type": "text/plain", "data": "Plain text content..."}
```

Here's the source structure for a **pdf**.

```js
{"type": "base64", "media_type": "application/pdf", "data": b64_enc_data}
```

Finally, here's the source structure for a **custom** document.

```js
{
  "type": "content",
  "content": [
    {"type": "text", "text": "First chunk"},
    {"type": "text", "text": "Second chunk"}
  ]
}
```

In [ ]:
#| export
def mk_ant_doc(content, title=None, context=None, citation=True, **kws):
    "Create an Anthropic document."
    if _is_pdf(content): src = {"type":"base64", "media_type":"application/pdf", "data":_mk_pdf(content)}
    elif isinstance(content,list): src = {"type":"content", "content":content}
    else: src = {"type":"text", "media_type":"text/plain", "data":content}
    return {"type":"document", "source":src, "citations":{"enabled":citation}, "title":title, "context":context, **kws}

Here's how you would implement the example from the citation's [docs](https://docs.anthropic.com/en/docs/build-with-claude/citations).

In [ ]:
doc = mk_ant_doc("The grass is green. The sky is blue.", title="My Document", context="This is a trustworthy document.")
mk_msg([doc, "What color is the grass and sky?"])

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()